$\color{blue}{\text{ Assignment 1:  Segmenting and Clustering Neighborhoods in Toronto}}$ 

### 1. Start by creating a new Notebook for this assignment.

In [20]:
#Installing lxml to read the html-file
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4
!pip install geocoder 

     |████████████████████████████████| 102kB 20.0MB/s ta 0:00:01
     |████████████████████████████████| 829kB 37.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 23.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future


In [2]:
import pandas as pd
import html5lib
import lxml


### 2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [3]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(wiki_url) # dfs is a list and its first element is the dataframe
#Toronto_df = pd.concat(dfs)
Toronto_df=dfs[0]
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In the following lines, I check if the table is imported correctly ...

In [4]:
type(Toronto_df)

pandas.core.frame.DataFrame

In [5]:
Toronto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
Postcode         288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 6.9+ KB


In [6]:
Toronto_df.shape

(288, 3)

### 3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
#first 5 rows of the dataframe
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
Toronto_df = Toronto_df[Toronto_df.Borough != 'Not assigned']

In [9]:
Toronto_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [10]:
Toronto_df=Toronto_df.groupby('Postcode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))

In [11]:
Toronto_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [12]:
Toronto_df['Neighbourhood'] = Toronto_df.apply(lambda x: x['Borough'] if x['Neighbourhood']=='Not assigned' else x['Neighbourhood'] , axis=1)

In [13]:
Toronto_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

To be sure, I checked if there exist any duplicate row in the dataframe

In [14]:
Toronto_df = Toronto_df.drop_duplicates()

I sorted the values of the column alphabetically according to the column Borough

In [28]:
Toronto_df.sort_values('Borough')

,Postcode,Borough,Neighbourhood
47,M4S,Central Toronto,Davisville
45,M4P,Central Toronto,Davisville North
44,M4N,Central Toronto,Lawrence Park
48,M4T,Central Toronto,"Summerhill East, Moore Park"
49,M4V,Central Toronto,"Summerhill West, Forest Hill SE, Deer Park, So..."
64,M5P,Central Toronto,"Forest Hill West, Forest Hill North"
46,M4R,Central Toronto,North Toronto West
63,M5N,Central Toronto,Roselawn
65,M5R,Central Toronto,"Yorkville, The Annex, North Midtown"
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown"


,Postcode,Borough,Neighbourhood
47,M4S,Central Toronto,Davisville
63,M5N,Central Toronto,Roselawn
46,M4R,Central Toronto,North Toronto West
64,M5P,Central Toronto,"Forest Hill West, Forest Hill North"
65,M5R,Central Toronto,"Yorkville, The Annex, North Midtown"
48,M4T,Central Toronto,"Summerhill East, Moore Park"
44,M4N,Central Toronto,Lawrence Park
45,M4P,Central Toronto,Davisville North
49,M4V,Central Toronto,"Summerhill West, Forest Hill SE, Deer Park, So..."
50,M4W,Downtown Toronto,Rosedale


- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [17]:
Toronto_df.shape

(103, 3)

- Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [25]:
Geo_df=pd.read_csv("http://cocl.us/Geospatial_data")

In [27]:
Geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


- The column of Toronto_df for the postal code is "Postcode" and for the dataframe Geo_df it is called "Postal Code". So we have to rename it, in order to eb able to merge the both dataframe together

In [35]:
Geo_df=Geo_df.rename(columns={"Postal Code": "Postcode"})
Geo_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
Toronto_Neighbourhood_df = pd.merge(Toronto_df, Geo_df, on='Postcode')

Toronto_Neighbourhood_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4S,Central Toronto,Davisville,43.704324,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M5P,Central Toronto,"Forest Hill West, Forest Hill North",43.696948,-79.411307
4,M5R,Central Toronto,"Yorkville, The Annex, North Midtown",43.672710,-79.405678
5,M4T,Central Toronto,"Summerhill East, Moore Park",43.689574,-79.383160
6,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
7,M4P,Central Toronto,Davisville North,43.712751,-79.390197
8,M4V,Central Toronto,"Summerhill West, Forest Hill SE, Deer Park, So...",43.686412,-79.400049
9,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529


In [41]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_Neighbourhood_df['Borough'].unique()),
        Toronto_Neighbourhood_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


We do not Postcode for the dataframe in teh map

In [59]:
neighborhoods=Toronto_Neighbourhood_df.drop(columns=['Postcode'])
neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Central Toronto,Davisville,43.704324,-79.388790
1,Central Toronto,Roselawn,43.711695,-79.416936
2,Central Toronto,North Toronto West,43.715383,-79.405678
3,Central Toronto,"Forest Hill West, Forest Hill North",43.696948,-79.411307
4,Central Toronto,"Yorkville, The Annex, North Midtown",43.672710,-79.405678


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>Toronto_explorer</em>, as shown below.

In [43]:
from geopy.geocoders import Nominatim

In [45]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top.

In [47]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [61]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

 let's simplify the above map and segment and cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [66]:
Toronto_data = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")]
Toronto_data

,Borough,Neighbourhood,Latitude,Longitude
0,Central Toronto,Davisville,43.704324,-79.388790
1,Central Toronto,Roselawn,43.711695,-79.416936
2,Central Toronto,North Toronto West,43.715383,-79.405678
3,Central Toronto,"Forest Hill West, Forest Hill North",43.696948,-79.411307
4,Central Toronto,"Yorkville, The Annex, North Midtown",43.672710,-79.405678
5,Central Toronto,"Summerhill East, Moore Park",43.689574,-79.383160
6,Central Toronto,Lawrence Park,43.728020,-79.388790
7,Central Toronto,Davisville North,43.712751,-79.390197
8,Central Toronto,"Summerhill West, Forest Hill SE, Deer Park, So...",43.686412,-79.400049
9,Downtown Toronto,Rosedale,43.679563,-79.377529


 I decided to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data

In [68]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [71]:
CLIENT_ID = 'MFAV1TEFM3VS1ZAD4AIOXUPKYZDNUZCLE235SQETXOSOWM0X' # your Foursquare ID
CLIENT_SECRET = '1NMKVB04Y43XSTTPD0OLFHOTPVKNMHZXLT4H5B4L1O3DNQXY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET )

Your credentails:
CLIENT_ID: MFAV1TEFM3VS1ZAD4AIOXUPKYZDNUZCLE235SQETXOSOWM0X
CLIENT_SECRET:1NMKVB04Y43XSTTPD0OLFHOTPVKNMHZXLT4H5B4L1O3DNQXY


### Let's explore the first neighborhood in our dataframe.

In [73]:
#Get the neighborhood's name.

Toronto_data.loc[0, 'Neighbourhood']

'Davisville'

In [75]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Davisville are 43.7043244, -79.3887901.


### let's get the top 100 venues that are in Marble Hill within a radius of 500 meters

In [76]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=MFAV1TEFM3VS1ZAD4AIOXUPKYZDNUZCLE235SQETXOSOWM0X&client_secret=1NMKVB04Y43XSTTPD0OLFHOTPVKNMHZXLT4H5B4L1O3DNQXY&v=20180605&ll=43.7043244,-79.3887901&radius=500&limit=100'

In [78]:
import requests

In [79]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d51b2106e465000334e3b37'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Davisville',
  'headerFullLocation': 'Davisville, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 34,
  'suggestedBounds': {'ne': {'lat': 43.7088244045, 'lng': -79.38257691798016},
   'sw': {'lat': 43.699824395499995, 'lng': -79.39500328201983}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ae6ea6ef964a52082a721e3',
       'name': 'Jules Cafe Patisserie',
       'location': {'address': '617 Mt Pleasant Ave',
        'crossStreet': 'at Manor Rd E',
        'lat': 43.70413799694304,
        'lng': -79.38841260442167,
        'labeledLatLngs':

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [80]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [82]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [83]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jules Cafe Patisserie,Dessert Shop,43.704138,-79.388413
1,Thobors Boulangerie Patisserie Café,Café,43.704514,-79.388616
2,XO Gelato,Dessert Shop,43.705177,-79.388793
3,Marigold Indian Bistro,Indian Restaurant,43.702881,-79.388008
4,Viva Napoli,Pizza Place,43.705752,-79.389125


In [84]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

34 venues were returned by Foursquare.


# Explore Neighborhoods in Toronto

Let's create a function to repeat the same process to all the neighborhoods in

In [85]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Now write the code to run the above function on each neighborhood and create a new dataframe called Toronto_venues.

In [87]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Davisville
Roselawn
North Toronto West
Forest Hill West, Forest Hill North
Yorkville, The Annex, North Midtown
Summerhill East, Moore Park
Lawrence Park
Davisville North
Summerhill West, Forest Hill SE, Deer Park, South Hill, Rathnelly
Rosedale
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
Berczy Park
Central Bay Street
Adelaide, Richmond, King
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
St. James Town
St. James Town, Cabbagetown
Kensington Market, Grange Park, Chinatown
University of Toronto, Harbord
Christie
Harbourfront East, Toronto Islands, Union Station
Stn A PO Boxes 25 The Esplanade
South Niagara, Bathurst Quay, King and Spadina, Railway Lands, CN Tower, Harbourfront West, Island airport
Underground city, First Canadian Place
Business Reply Mail Processing Centre 969 Eastern
Studio District
The Beaches West, India Bazaar
Riverdale, The Danforth West
The Beaches
Parkdale, Roncesvalles
High Park, The Junction South
Exhibition 

In [88]:
#Let's check the size of the resulting dataframe
print(Toronto_venues.shape)
Toronto_venues.head()

(1690, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Davisville,43.704324,-79.38879,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
1,Davisville,43.704324,-79.38879,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café
2,Davisville,43.704324,-79.38879,XO Gelato,43.705177,-79.388793,Dessert Shop
3,Davisville,43.704324,-79.38879,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
4,Davisville,43.704324,-79.38879,Viva Napoli,43.705752,-79.389125,Pizza Place


Let's check how many venues were returned for each neighborhood


In [90]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, Richmond, King",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
Central Bay Street,82,82,82,82,82,82
Christie,16,16,16,16,16,16
Church and Wellesley,82,82,82,82,82,82
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34
Davisville North,10,10,10,10,10,10


Let's find out how many unique categories can be curated from all the returned venues

In [91]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


## 3. Analyze Each Neighborhood

In [93]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
Toronto_onehot.shape

(1690, 232)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [98]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, Richmond, King",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.000000,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000
2,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.012195,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012195,...,0.000000,0.00,0.000000,0.000000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000
4,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.012195,0.012195,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012195,...,0.012195,0.00,0.000000,0.000000,0.00,0.000000,0.012195,0.012195,0.000000,0.012195
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,...,0.000000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.000000,0.00,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.010000,0.000000


In [99]:
Toronto_grouped.shape

(38, 232)

## Let's print each neighborhood along with the top 5 most common venues

In [101]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, Richmond, King----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4              Bar  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0            Spa  0.06
1     Restaurant  0.06
2  Burrito Place  0.06
3     Smoke Shop  0.06
4     Skate Park  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2      Ice Cream Shop  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.04


----Christie----
           venue  freq
0  Grocery Store  0.19
1           Café  0.19
2           Park  0.12
3     Restaurant  0.06
4     Baby Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coff

### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order

In [102]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [104]:
import numpy as np

In [107]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, Richmond, King",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Hotel,Restaurant,Gym,Asian Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Beer Bar,Bakery,Steakhouse,Seafood Restaurant,Cheese Shop,Farmers Market,Restaurant
2,Business Reply Mail Processing Centre 969 Eastern,Gym / Fitness Center,Auto Workshop,Park,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop,Fast Food Restaurant
3,Central Bay Street,Coffee Shop,Café,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Middle Eastern Restaurant,Bubble Tea Shop,Chinese Restaurant,Spa
4,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Diner,Athletics & Sports,Restaurant,Nightclub,Convenience Store


### 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [109]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [119]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [121]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster,Cluster Labele,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Davisville,43.704324,-79.388790,0,0,0,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Indian Restaurant,Japanese Restaurant,Farmers Market
1,Central Toronto,Roselawn,43.711695,-79.416936,1,1,1,Garden,Home Service,Music Venue,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
2,Central Toronto,North Toronto West,43.715383,-79.405678,0,0,0,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Bagel Shop,Gift Shop,Metro Station,Mexican Restaurant,Diner,Dessert Shop
3,Central Toronto,"Forest Hill West, Forest Hill North",43.696948,-79.411307,4,4,4,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,Central Toronto,"Yorkville, The Annex, North Midtown",43.672710,-79.405678,0,0,0,Café,Sandwich Place,Coffee Shop,Pizza Place,Burger Joint,Indian Restaurant,Jewish Restaurant,Pub,American Restaurant,BBQ Joint


In [123]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [126]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


### 5. Examine Clusters

### Cluster 1

In [128]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labele,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,0,0,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Indian Restaurant,Japanese Restaurant,Farmers Market
2,North Toronto West,0,0,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Bagel Shop,Gift Shop,Metro Station,Mexican Restaurant,Diner,Dessert Shop
4,"Yorkville, The Annex, North Midtown",0,0,Café,Sandwich Place,Coffee Shop,Pizza Place,Burger Joint,Indian Restaurant,Jewish Restaurant,Pub,American Restaurant,BBQ Joint
7,Davisville North,0,0,Gym / Fitness Center,Hotel,Clothing Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Asian Restaurant,Park,Gym,Grocery Store
8,"Summerhill West, Forest Hill SE, Deer Park, So...",0,0,Coffee Shop,Pub,American Restaurant,Liquor Store,Sports Bar,Bagel Shop,Supermarket,Restaurant,Sushi Restaurant,Fried Chicken Joint
10,Church and Wellesley,0,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Gastropub,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant
11,"Harbourfront, Regent Park",0,0,Coffee Shop,Bakery,Park,Café,Breakfast Spot,Pub,Mexican Restaurant,Farmers Market,Shoe Store,Brewery
12,"Ryerson, Garden District",0,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Restaurant,Pizza Place,Diner,Ramen Restaurant,Fast Food Restaurant
13,Berczy Park,0,0,Coffee Shop,Cocktail Bar,Café,Beer Bar,Bakery,Steakhouse,Seafood Restaurant,Cheese Shop,Farmers Market,Restaurant
14,Central Bay Street,0,0,Coffee Shop,Café,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Middle Eastern Restaurant,Bubble Tea Shop,Chinese Restaurant,Spa


### Cluster 2

In [129]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labele,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Roselawn,1,1,Garden,Home Service,Music Venue,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### Cluster 3

In [130]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labele,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Lawrence Park,2,2,Photography Studio,Park,Bus Line,Swim School,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


### Cluster 4

In [131]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labele,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Summerhill East, Moore Park",3,3,Playground,Trail,Summer Camp,Wings Joint,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


### Cluster 5

In [132]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labele,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Forest Hill West, Forest Hill North",4,4,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
9,Rosedale,4,4,Park,Playground,Trail,Building,Wings Joint,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
